# 要約 
このJupyter Notebookは、Kaggleの「20の質問 LLM コンペティション」に参加するための問題解決手法を示しています。このコンペティションの目標は、AIが秘密の単語を当てるために最大20の質問に答える「20の質問」ゲームをプレイすることです。

ノートブックでは、具体的には以下の内容に取り組んでいます：

1. **問題設定**: 参加者が考えた秘密の単語を、質問者LLMが質問を通じて推測するという構造のゲームです。このゲームは、限られた質問回数の中でターゲット単語を当てるために戦略的かつ効率的な情報収集を行う必要があります。

2. **使用技術**:
   - **ライブラリ**: `transformers`ライブラリを使用し、事前学習済みのRoBERTaモデル（"deepset/roberta-base-squad2"）を利用して、質問応答タスクを実行します。
   - **GPU利用**: 質問応答パイプラインは、GPUを利用して処理速度を向上させています。

3. **手法**:
   - **プロンプト生成**: `generate_prompt`関数が、キーワードとそのカテゴリー、大陸などのコンテキスト情報をもとにプロンプトを生成します。
   - **応答生成**: `get_answer_from_model`関数を使用して、生成したプロンプトに対する質問の回答をモデルから取得します。

4. **実行**: `main`関数内で実際にキーワード「Mount Everest」を使用し、関連する質問をリスト化して、それぞれの質問に対するモデルの回答を表示します。これにより、モデルが受け取った質問に対してどのように応答するかが確認できます。

このノートブックは、言語モデルを利用して「20の質問」ゲームに取り組むための基礎を築くものであり、特に自然言語処理の技術を用いて、AIが人間のゲームプレイのように推論する能力を示しています。

---


# 用語概説 
以下に示すのは、上記のJupyter Notebookにおいて初心者がつまずきそうな専門用語の簡単な解説です。

### 専門用語の解説

1. **RoBERTa (Robustly optimized BERT approach)**:
   - BERTに基づいた自然言語処理モデルで、事前学習を強化し、いくつかのハイパーパラメータやトレーニングテクニックを最適化している。QA (質問応答) タスクに特化したファインチューニングが施されている。

2. **トークナイザー**:
   - テキストを入力として受け取り、それをモデルが理解できる形式（トークン）に変換するツール。単語や文の区切りを認識し、ID番号に変換する。

3. **パイプライン**:
   - 特定のタスク（例: 質問応答）を実行するための簡易的なインターフェース。モデルのロード、トークン化、入力の生成、出力の取得を一連の処理としてまとめている。

4. **ファインチューニング**:
   - 事前学習されたモデルを特定のデータセットやタスクに対して最適化するプロセス。通常、大規模なデータセットで学習したモデルを用いて、より少ないデータセットで再学習すること。

5. **プロンプト**:
   - モデルに入力するための情報や質問の表現。コンテキスト情報も含めてモデルに提供することで、適切な応答を得るための手がかりを与える。

6. **コンテキスト**:
   - モデルが質問に答えるために使用する情報のこと。通常は、モデルが質問を理解し応答するための背景情報を指す。

7. **GPU (Graphics Processing Unit)**:
   - 大量の計算を並列処理するために特化したプロセッサ。深層学習モデルのトレーニングや推論において計算速度を大幅に向上させるために使用される。

8. **除外情報 (Negate)**:
   - 特定の条件や選択肢から排除する情報。特定のカテゴリに含まれないことを示すことで、モデルの推測や質問の精度を高めるために用いられる。

9. **応答 (Response)**:
   - モデルが質問に基づいて生成した答えのこと。この場合、質問応答タスクにおいて生成された具体的な情報を指す。

10. **エピソード**:
    - コンペティションの進行中に行われる1回のゲームセッション。ゲームのルールに従い、質問と応答を繰り返すことで進行する。

これらの用語の理解を深めることで、ノートブックに記載された内容や全体的なコンペティションのアプローチをよりよく理解できると思います。

---


### 「20の質問 LLM コンペティション」は、人工知能、自然言語処理、ゲーム理論の分野を結びつける革新的な挑戦です。このコンペティションは、参加者の一人が秘密の単語を考え、他の参加者が最大20のはい/いいえの質問を通じてそれを推測するという古典的な「20の質問」ゲームを中心に展開されます。このシンプルでありながら魅力的なゲームが、高度なAIコンペティションの基盤となっています。

<figure>
        <img src="https://www.kaggle.com/competitions/61247/images/header" alt ="Audio Art" style='width:800px;height:500px;'>
        <figcaption>


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# 事前学習済みのRoBERTaモデルとトークナイザーをロードします
model_id = "deepset/roberta-base-squad2"  # QA用にファインチューニングされたRoBERTaモデル
tokenizer = AutoTokenizer.from_pretrained(model_id)  # トークナイザーの初期化
model = AutoModelForQuestionAnswering.from_pretrained(model_id)  # モデルの初期化

# GPU用の質問応答パイプラインを初期化します
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)  # device=0はGPUを使用することを確認します

def generate_prompt(keyword, category, negate, continent):
    if category in negate:
        prompt = (f"We are playing 20 questions. The keyword is {keyword}. It is a {category}. {negate[category]} "
                  f"This word has first letter {keyword[0]}. This {category} is located in {continent}.")
    else:
        prompt = (f"We are playing 20 questions. The keyword is {keyword}. It is a thing. It is not a city. "
                  f"It is not a country. It is not a landmark. This word has first letter {keyword[0]}.")
    return prompt  # 生成したプロンプトを返します

def get_answer_from_model(prompt, question):
    # モデルから回答を生成します
    response = qa_pipeline(question=question, context=prompt)  # 質問とプロンプトを使用して応答を取得
    answer = response['answer']  # 応答から答えを取得します
    return answer  # 答えを返します

def main():
    keyword = "Mount Everest"  # キーワードを設定します（ここではエベレスト）
    category = "mountain"  # カテゴリーを設定します（ここでは山）
    negate = {"mountain": "It is not a country, city, or person."}  # 除外情報を設定します
    continent = "Asia"  # 大陸を設定します（ここではアジア）

    prompt = generate_prompt(keyword, category, negate, continent)  # プロンプトを生成します

    questions = [
        "Is this a country or city?",  # 質問リストの最初の質問
        "No, it is a mountain. What is the name of this mountain?",  # 次の質問
        "Where is this place?"  # 最後の質問
    ]

    # 構築したプロンプトを表示します
    print(f"Prompt:\n{prompt}\n")

    # 各質問を繰り返します
    for question in questions:
        # モデルから答えを取得します
        answer = get_answer_from_model(prompt, question)
        
        # フォーマットされた出力を表示します
        print(f"Question: '{question}'")
        print(f"Answer: '{answer}'\n")

if __name__ == "__main__":
    main()  # メイン関数を実行します

In [ ]:
code = """
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# 事前学習済みのRoBERTaモデルとトークナイザーをロードします
model_id = "deepset/roberta-base-squad2"  # QA用にファインチューニングされたRoBERTaモデル
tokenizer = AutoTokenizer.from_pretrained(model_id)  # トークナイザーの初期化
model = AutoModelForQuestionAnswering.from_pretrained(model_id)  # モデルの初期化

# GPU用の質問応答パイプラインを初期化します
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)  # device=0はGPUを使用することを確認します

def generate_prompt(keyword, category, negate, continent):
    if category in negate:
        prompt = (f"We are playing 20 questions. The keyword is {keyword}. It is a {category}. {negate[category]} "
                  f"This word has first letter {keyword[0]}. This {category} is located in {continent}.")
    else:
        prompt = (f"We are playing 20 questions. The keyword is {keyword}. It is a thing. It is not a city. "
                  f"It is not a country. It is not a landmark. This word has first letter {keyword[0]}.")
    return prompt  # 生成したプロンプトを返します

def get_answer_from_model(prompt, question):
    # モデルから回答を生成します
    response = qa_pipeline(question=question, context=prompt)  # 質問とプロンプトを使用して応答を取得
    answer = response['answer']  # 応答から答えを取得します
    return answer  # 答えを返します

def main():
    keyword = "Mount Everest"  # キーワードを設定します（ここではエベレスト）
    category = "mountain"  # カテゴリーを設定します（ここでは山）
    negate = {"mountain": "It is not a country, city, or person."}  # 除外情報を設定します
    continent = "Asia"  # 大陸を設定します（ここではアジア）

    prompt = generate_prompt(keyword, category, negate, continent)  # プロンプトを生成します

    questions = [
        "Is this a country or city?",  # 質問リストの最初の質問
        "No, it is a mountain. What is the name of this mountain?",  # 次の質問
        "Where is this place?"  # 最後の質問
    ]

    # 構築したプロンプトを表示します
    print(f"Prompt:\n{prompt}\n")

    # 各質問を繰り返します
    for question in questions:
        # モデルから答えを取得します
        answer = get_answer_from_model(prompt, question)
        
        # フォーマットされた出力を表示します
        print(f"Question: '{question}'")
        print(f"Answer: '{answer}'\n")

if __name__ == "__main__":
    main()  # メイン関数を実行します
"""

# コードをsubmission.pyに保存します
with open("/kaggle/working/submission.py", "w") as f:
    f.write(code)  # コードを書き込む